# Part 1: Baseline - Understanding Batch Inference for Paper Evaluation

In this workshop, we'll build a simple paper evaluation system that:
1. **Fetches** recent AI research papers from arXiv
2. **Evaluates** each paper's relevance using the Batch API
3. **Ranks** papers to find the most interesting ones for our team

This is our **baseline** - a fixed pipeline with no decision-making.
In later parts, we'll transform this into a true AI agent.

---

## Setup

First, let's import our libraries and configure the environment.

**What we're using:**
- `arxiv` - Python client for searching arXiv papers
- `openai` - Client for the OpenAI-compatible Batch API. We will be using the Doubleword Batch API in our example.
- `dotenv` - Load API keys from `.env` file

In [2]:
# Standard library imports
import os
import sys
import json
import time
import re
from datetime import datetime, timedelta
from pathlib import Path

# Third-party imports
import arxiv
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Add src directory to path so we can import our modules
sys.path.insert(0, str(Path.cwd().parent / 'src'))

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


## Get our Doubleword API Key

Now we need to create a Doubleword API Key. 

1. Sign up at https://app.doubleword.com
2. Create an API key, save this in a safe place
3. Copy the `.env.example` file in the same directory, name it `.env` and add your api key 

In [3]:
# Configuration - these come from your .env file
API_KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = os.getenv("OPENAI_BASE_URL")  # Optional: for non-OpenAI providers
MODEL_NAME = os.getenv("MODEL_NAME", "gpt-4o-mini")

# Verify we have an API key
if not API_KEY:
    print("⚠️  Warning: OPENAI_API_KEY not found in environment!")
    print("   Create a .env file with your API key, or we'll use sample data.")
else:
    print(f"✓ API key loaded (ends with ...{API_KEY[-4:]})")
    print(f"✓ Model: {MODEL_NAME}")
    if BASE_URL:
        print(f"✓ Base URL: {BASE_URL}")

✓ API key loaded (ends with ...1Ok4)
✓ Model: Qwen/Qwen3-VL-30B-A3B-Instruct-FP8
✓ Base URL: https://api.doubleword.ai/v1


Now we can run a quick test to make sure that we have established connectivity to Doubleword. The Doubleword follows the openai spec, so let's see which models we have availabile. 

In [4]:
from openai import OpenAI

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

print("Available models:")
for model in client.models.list():
    print(f"  • {model.id}")

Available models:
  • Qwen/Qwen3-VL-235B-A22B-Instruct-FP8
  • Qwen/Qwen3-Embedding-8B
  • Qwen/Qwen3-VL-30B-A3B-Instruct-FP8


### What We're Building

Our goal is to help an AI engineering team stay current with research.
Every day, hundreds of papers are published - we can't read them all!

**The Pipeline:**
```
arXiv API → Fetch Papers → Batch API → Evaluate Relevance → Ranked List
```

**Why Batch API?**
- 50% cheaper than real-time API calls
- Process many papers in parallel
- Perfect for non-interactive workloads

---

## Step 1: Fetch Papers from arXiv

arXiv is a free repository of research papers, widely used in AI/ML.
We'll search for recent papers matching our keywords.

Let's look at the code that fetches papers:

In [10]:
def get_papers(keywords: list[str], max_results: int = 20, days_back: int = 7) -> list[dict]:
    """
    Fetch recent papers from arXiv matching the given keywords.
    
    Args:
        keywords: Search terms (e.g., ["LLM", "transformers"])
        max_results: Maximum papers to return
        days_back: How far back to search
    
    Returns:
        List of paper dictionaries
    """
    
    # Calculate cutoff date
    cutoff_date = datetime.now() - timedelta(days=days_back)
    
    # Build search query: "keyword1" OR "keyword2" OR ...
    # Quotes ensure exact phrase matching
    query = " OR ".join([f'"{kw}"' for kw in keywords])
    print(f"🔍 Searching arXiv for: {query}")
    print(f"📅 Looking for papers since: {cutoff_date.strftime('%Y-%m-%d')}")
    
    # Configure the arXiv search
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,  # Most recent first
        sort_order=arxiv.SortOrder.Descending
    )
    
    # Execute search and collect results
    client = arxiv.Client()
    papers = []
    
    for paper in client.results(search):
        # Only include papers from our time window
        paper_date = paper.published.replace(tzinfo=None)
        if paper_date >= cutoff_date:
            papers.append({
                'id': paper.entry_id.split('/')[-1],  # e.g., "2401.12345v1"
                'title': paper.title,
                'authors': [author.name for author in paper.authors],
                'abstract': paper.summary,
                'published': paper.published.isoformat(),
                'url': paper.entry_id
            })
    
    print(f"✓ Found {len(papers)} papers")
    return papers

### Option A: Fetch Live Papers from arXiv

This searches arXiv in real-time. Results vary based on what's been published recently. You can tweak `max_results` to get more or less papers to search from

In [11]:
# Define what we're interested in
SEARCH_KEYWORDS = [
    "large language models",
    "LLM agents",
    "prompt engineering"
]

# Fetch papers (set to False to use sample data instead)
USE_LIVE_DATA = False  # Change to True to fetch from arXiv

if USE_LIVE_DATA:
    papers = get_papers(SEARCH_KEYWORDS, max_results=20)
else:
    print("📂 Using sample data (set USE_LIVE_DATA = True to fetch live)")

📂 Using sample data (set USE_LIVE_DATA = True to fetch live)


### Option B: Load Sample Papers (Recommended for Workshop)

For consistent results during the workshop, we'll use pre-saved sample papers.
This also works offline and doesn't require waiting for arXiv.

In [13]:
def load_sample_papers(filepath: str = None) -> list[dict]:
    """
    Load pre-saved sample papers for offline demos.
    """
    if filepath is None:
        # Default path relative to notebooks directory
        filepath = Path.cwd().parent / "sample_data" / "papers.json"
    
    with open(filepath, 'r') as f:
        papers = json.load(f)
    
    print(f"✓ Loaded {len(papers)} sample papers")
    return papers

# Load sample papers if we didn't fetch live ones
if not USE_LIVE_DATA:
    papers = load_sample_papers()

✓ Loaded 20 sample papers


### Explore the Papers

Let's see what we got back from arXiv (or our sample data).

In [14]:
# How many papers do we have?
print(f"Total papers: {len(papers)}")
print()

# Look at the structure of one paper
print("📄 Structure of a paper object:")
print(f"   Keys: {list(papers[0].keys())}")

Total papers: 20

📄 Structure of a paper object:
   Keys: ['id', 'title', 'authors', 'abstract', 'published', 'url']


In [15]:
# Display the first 5 papers
print("="*80)
print("FIRST 5 PAPERS")
print("="*80)

for i, paper in enumerate(papers[:5], 1):
    print(f"\n{i}. {paper['title'][:70]}..." if len(paper['title']) > 70 else f"\n{i}. {paper['title']}")
    print(f"   ID: {paper['id']}")
    print(f"   Authors: {', '.join(paper['authors'][:3])}{'...' if len(paper['authors']) > 3 else ''}")
    print(f"   Abstract: {paper['abstract'][:150]}...")

FIRST 5 PAPERS

1. Efficient Inference for Large Language Models: A Survey
   ID: 2401.00001v1
   Authors: Alice Chen, Bob Williams, Carol Davis
   Abstract: Large Language Models (LLMs) have revolutionized natural language processing but face significant challenges in deployment due to their computational ...

2. ReAct Agents: Combining Reasoning and Acting in Language Models
   ID: 2401.00002v1
   Authors: David Park, Emily Zhang
   Abstract: We present an improved framework for building AI agents that interleave reasoning traces with actions. Our approach enables language models to generat...

3. Prompt Engineering Best Practices for Production Systems
   ID: 2401.00003v1
   Authors: Frank Miller, Grace Lee, Henry Wang
   Abstract: As large language models become integral to production systems, effective prompt engineering has emerged as a critical skill. This paper presents a sy...

4. Scaling Laws for Fine-tuning Language Models
   ID: 2401.00004v1
   Authors: Isabella Rodriguez, 

### What We Got Back

Each paper is a dictionary with:
- **id**: arXiv identifier (useful for linking back to papers)
- **title**: The paper's title
- **authors**: List of author names
- **abstract**: Summary of the paper (this is what we'll evaluate!)
- **published**: When it was submitted to arXiv
- **url**: Direct link to the paper

Now the question is: **which of these 20 papers should our team actually read?**

---

## Step 2: Batch Evaluation with AI

Now we'll use an LLM to evaluate each paper's relevance to our team.

### The Team Profile

First, we define what our team cares about. This is the "context" the AI uses to judge relevance.

In [14]:
# Define what makes a paper "relevant" for our team
# Try modifying this to see how it changes the results!

TEAM_PROFILE = {
    "focus": """
        The team is building AI-powered applications and wants to stay
        current with the latest research on language models, inference
        optimization, and practical AI engineering.
    """,
    
    "interests": [
        "Large language model architectures and improvements",
        "Inference optimization and cost reduction",
        "Prompt engineering and techniques",
        "AI agents and tool use",
        "Evaluation methods for LLMs",
    ],
    
    "avoid": [
        "Pure theoretical papers without practical applications",
        "Incremental benchmark improvements",
        "Papers focused only on training from scratch",
    ]
}

print("Team Focus:")
print(TEAM_PROFILE['focus'].strip())
print("\nInterests:")
for interest in TEAM_PROFILE['interests']:
    print(f"  • {interest}")

Team Focus:
The team is building AI-powered applications and wants to stay
        current with the latest research on language models, inference
        optimization, and practical AI engineering.

Interests:
  • Large language model architectures and improvements
  • Inference optimization and cost reduction
  • Prompt engineering and techniques
  • AI agents and tool use
  • Evaluation methods for LLMs


### The Evaluation Prompt

This is the prompt we send to the LLM for each paper.
Notice how we:
1. Give context about the team
2. Provide the paper details
3. Ask for structured JSON output

In [15]:
def create_evaluation_prompt(paper: dict, team_profile: dict) -> str:
    """
    Create the prompt that asks the AI to evaluate a paper.
    
    This is where the "magic" happens - we give the model:
    1. Context about what the team cares about
    2. The paper's title and abstract
    3. Clear instructions on how to respond
    """
    
    # Format interests and avoid lists as bullet points
    interests_text = "\n".join(f"  - {item}" for item in team_profile['interests'])
    avoid_text = "\n".join(f"  - {item}" for item in team_profile['avoid'])
    
    prompt = f"""You are evaluating research papers for an AI engineering team.

TEAM PROFILE:
{team_profile['focus'].strip()}

What they find valuable:
{interests_text}

What to avoid recommending:
{avoid_text}

---

PAPER TO EVALUATE:

Title: {paper['title']}

Abstract:
{paper['abstract']}

---

INSTRUCTIONS:
Score this paper's relevance to the team on a scale of 0-10.
- 0-3: Not relevant
- 4-6: Somewhat relevant
- 7-10: Highly relevant, team should read this

Respond with ONLY valid JSON in this exact format:
{{
    "relevance_score": <integer 0-10>,
    "key_insight": "<one sentence explaining the main takeaway>",
    "why_relevant": "<one sentence explaining why this score>"
}}"""
    
    return prompt

# Let's see what a prompt looks like
sample_prompt = create_evaluation_prompt(papers[0], TEAM_PROFILE)
print("📝 SAMPLE PROMPT (first paper):")
print("="*80)
print(sample_prompt[:1500] + "..." if len(sample_prompt) > 1500 else sample_prompt)

📝 SAMPLE PROMPT (first paper):
You are evaluating research papers for an AI engineering team.

TEAM PROFILE:
The team is building AI-powered applications and wants to stay
        current with the latest research on language models, inference
        optimization, and practical AI engineering.

What they find valuable:
  - Large language model architectures and improvements
  - Inference optimization and cost reduction
  - Prompt engineering and techniques
  - AI agents and tool use
  - Evaluation methods for LLMs

What to avoid recommending:
  - Pure theoretical papers without practical applications
  - Incremental benchmark improvements
  - Papers focused only on training from scratch

---

PAPER TO EVALUATE:

Title: Efficient Inference for Large Language Models: A Survey

Abstract:
Large Language Models (LLMs) have revolutionized natural language processing but face significant challenges in deployment due to their computational requirements. This survey comprehensively reviews rece

### Creating the Batch File

The Batch API expects a JSONL file (one JSON object per line).
Each line represents one request to the API. We can either build the JSONl file using `create_batch_file` below or we can use [autobatcher](https://docs.doubleword.ai/batches/autobatcher)

#### Option A: Create Batch File

In [16]:
def create_batch_file(papers: list[dict], model: str, output_path: str = "batch_input.jsonl") -> str:
    """
    Create a JSONL file for the Batch API.
    
    Each line contains:
    - custom_id: Links response back to the paper
    - method: HTTP method (POST)
    - url: API endpoint
    - body: The actual request (model, messages, etc.)
    """
    
    with open(output_path, 'w') as f:
        for paper in papers:
            request = {
                "custom_id": paper['id'],  # We'll use this to match responses
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": model,
                    "max_tokens": 500,
                    "messages": [
                        {
                            "role": "user",
                            "content": create_evaluation_prompt(paper, TEAM_PROFILE)
                        }
                    ]
                }
            }
            f.write(json.dumps(request) + '\n')
    
    return output_path

# Create the batch file
batch_file = create_batch_file(papers, MODEL_NAME)
print(f"✓ Created batch file: {batch_file}")
print(f"  Contains {len(papers)} requests")

✓ Created batch file: batch_input.jsonl
  Contains 20 requests


In [17]:
# Let's peek at the batch file
print("📄 First 2 lines of batch file:")
print("="*80)

with open(batch_file, 'r') as f:
    for i, line in enumerate(f):
        if i >= 2:
            break
        data = json.loads(line)
        # Show a truncated version
        print(f"\nRequest {i+1}:")
        print(f"  custom_id: {data['custom_id']}")
        print(f"  method: {data['method']}")
        print(f"  model: {data['body']['model']}")
        print(f"  message length: {len(data['body']['messages'][0]['content'])} chars")

📄 First 2 lines of batch file:

Request 1:
  custom_id: 2401.00001v1
  method: POST
  model: Qwen/Qwen3-VL-30B-A3B-Instruct-FP8
  message length: 1799 chars

Request 2:
  custom_id: 2401.00002v1
  method: POST
  model: Qwen/Qwen3-VL-30B-A3B-Instruct-FP8
  message length: 1736 chars


### Submit to the Batch API

Now we:
1. Upload the batch file
2. Create a batch job
3. Wait for completion
4. Download results

**Note:** This requires a valid API key. If you don't have one, skip to the "Mock Results" section below.

In [ ]:
def submit_batch(client: OpenAI, input_file: str) -> str:
    """
    Upload file and create a batch job.
    Returns the batch ID for tracking.
    """
    
    # Step 1: Upload the file
    print("📤 Uploading batch file...")
    with open(input_file, 'rb') as f:
        uploaded_file = client.files.create(file=f, purpose="batch")
    print(f"   File ID: {uploaded_file.id}")
    
    # Step 2: Create the batch job
    print("🚀 Creating batch job...")
    batch = client.batches.create(
        input_file_id=uploaded_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h" # this can be 24hr or 1hr 
    )
    print(f"   Batch ID: {batch.id}")
    print(f"   Status: {batch.status}")
    
    return batch.id


def wait_for_batch(client: OpenAI, batch_id: str, poll_interval: int = 10) -> str:
    """
    Poll until batch completes, then return output file ID.
    """
    
    print(f"\n⏳ Waiting for batch to complete...")
    print("   (This typically takes 1-5 minutes)\n")
    
    while True:
        batch = client.batches.retrieve(batch_id)
        status = batch.status
        
        # Show progress
        completed = (batch.request_counts.completed or 0)
        failed = (batch.request_counts.failed or 0)
        total = batch.request_counts.total or 0
        
        print(f"   Status: {status} | Progress: {completed + failed}/{total}")
        
        if status == "completed":
            print("\n✓ Batch completed!")
            return batch.output_file_id
        elif status in ["failed", "expired", "cancelled"]:
            raise Exception(f"Batch failed with status: {status}")
        
        time.sleep(poll_interval)


def download_results(client: OpenAI, output_file_id: str) -> list[dict]:
    """
    Download and parse the batch results.
    """
    
    print(f"📥 Downloading results...")
    content = client.files.content(output_file_id)
    
    results = []
    for line in content.text.strip().split('\n'):
        if line:
            results.append(json.loads(line))
    
    print(f"   Downloaded {len(results)} results")
    return results

In [ ]:
# Run the batch evaluation
# Set this to True when you have a valid API key
RUN_BATCH_API = True

if RUN_BATCH_API and API_KEY:
    # Create the client
    client_kwargs = {'api_key': API_KEY}
    if BASE_URL:
        client_kwargs['base_url'] = BASE_URL
    
    client = OpenAI(**client_kwargs)
       
    # Submit and wait
    batch_id = submit_batch(client, batch_file)
    output_file_id = wait_for_batch(client, batch_id)
    raw_results = download_results(client, output_file_id)
    
    print("\n✓ Batch evaluation complete!")
else:
    print("⏭️  Skipping live API call (RUN_BATCH_API = False)")
    print("   We'll use mock results in the next cell.")


📤 Uploading batch file...
   File ID: e6e3dd08-933b-4ff4-b303-c567e5ba7eac
🚀 Creating batch job...
   Batch ID: f2f43151-5ee3-4668-ad57-72f514119beb
   Status: in_progress

⏳ Waiting for batch to complete...
   (This typically takes 1-5 minutes)

   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress: 0/20
   Status: in_progress | Progress

KeyboardInterrupt: 

### Option B: Use AutoBatcher 

In [21]:
# Using autobatcher for efficient batch processing
# Install: pip install autobatcher

import asyncio
from autobatcher import BatchOpenAI

async def evaluate_papers_with_autobatcher(papers, model, team_profile):
    """
    Evaluate papers using autobatcher - waits for 20 requests before sending.
    """
    
    # Create autobatcher client - batch_size=20 means it waits for 20 requests
    async with BatchOpenAI(
        api_key=API_KEY,
        base_url=BASE_URL,
        batch_size=20,              # Wait until we have 20 requests
        batch_window_seconds=60.0,  # Or send after 60 seconds if we don't hit 20
        poll_interval_seconds=5.0,  # Check for results every 5 seconds
    ) as client:
        
        # Create all evaluation tasks concurrently
        async def evaluate_single_paper(paper):
            prompt = create_evaluation_prompt(paper, team_profile)
            response = await client.chat.completions.create(
                model=model,
                max_tokens=500,
                messages=[{"role": "user", "content": prompt}]
            )
            content = response.choices[0].message.content
            evaluation = parse_json_response(content)
            if evaluation:
                evaluation['paper_id'] = paper['id']
            return evaluation
        
        # Send all 20 papers at once - autobatcher will batch them
        print(f"📤 Sending {len(papers)} papers to autobatcher (batch_size=20)...")
        tasks = [evaluate_single_paper(paper) for paper in papers]
        results = await asyncio.gather(*tasks)
        
        # Filter out any None results
        evaluations = [r for r in results if r is not None]
        print(f"✓ Received {len(evaluations)} evaluations")
        
        return evaluations
    
# Run the evaluation
print("🚀 Starting batch evaluation with autobatcher...")
evaluations = await evaluate_papers_with_autobatcher(papers, MODEL_NAME, TEAM_PROFILE)

2026-02-03 11:59:22.830 | DEBUG    | autobatcher.client:__init__:172 - Initialized with batch_size=20, window=60.0s
2026-02-03 11:59:22.831 | DEBUG    | autobatcher.client:_enqueue_request:200 - Starting 60.0s batch window timer
2026-02-03 11:59:22.832 | DEBUG    | autobatcher.client:_enqueue_request:208 - Batch size 20 reached


🚀 Starting batch evaluation with autobatcher...
📤 Sending 20 papers to autobatcher (batch_size=20)...
█████░░░░░░░░░░░░░░░  25% | Preparing batch (20 requests)...
██████████░░░░░░░░░░  50% | Uploading file...


2026-02-03 11:59:23.681 | DEBUG    | autobatcher.client:_submit_batch:277 - Uploaded batch file: a94cc72f-5825-4ac3-aeb1-ae7cb6160278


███████████████░░░░░  75% | Creating batch...


2026-02-03 11:59:24.112 | DEBUG    | autobatcher.client:_submit_batch:287 - Submitted batch d64e42bb-9f59-43ab-afc3-b0742af84bc2 with 20 requests
2026-02-03 11:59:24.113 | DEBUG    | autobatcher.client:_poll_batches:318 - Poller started with 1 active batches


████████████████████ 100% | Submitted ✓ (batch d64e42bb-9f5...)


2026-02-03 11:59:29.610 | DEBUG    | autobatcher.client:_poll_batches:329 - Batch d64e42bb-9f5 status: in_progress (completed=0/20)
2026-02-03 11:59:35.965 | DEBUG    | autobatcher.client:_poll_batches:329 - Batch d64e42bb-9f5 status: in_progress (completed=0/20)
2026-02-03 11:59:42.114 | DEBUG    | autobatcher.client:_poll_batches:329 - Batch d64e42bb-9f5 status: in_progress (completed=0/20)
2026-02-03 11:59:48.328 | DEBUG    | autobatcher.client:_poll_batches:329 - Batch d64e42bb-9f5 status: in_progress (completed=0/20)


CancelledError: 

### Mock Results (For Workshop Demo)

If you didn't run the actual API call, here are realistic mock results
so you can see how the rest of the pipeline works.

In [ ]:
# Mock results that simulate what the API would return
# These are pre-generated evaluations for our sample papers (REAL arXiv papers)

MOCK_EVALUATIONS = [
    {"paper_id": "2410.21276v1", "relevance_score": 10, 
     "key_insight": "Comprehensive survey of tool learning for LLM-based agents covering understanding, creation, and optimization.",
     "why_relevant": "Directly addresses AI agents and tool use - a primary team interest."},
    
    {"paper_id": "2310.04406v2", "relevance_score": 9,
     "key_insight": "AgentTuning improves LLM agent abilities while maintaining general capabilities through hybrid instruction-tuning.",
     "why_relevant": "Practical approach to building better AI agents."},
    
    {"paper_id": "2312.10997v3", "relevance_score": 7,
     "key_insight": "Framework for governing agentic AI systems with practical deployment guidelines.",
     "why_relevant": "Relevant for safe deployment but more governance-focused than technical."},
    
    {"paper_id": "2401.04088v2", "relevance_score": 9,
     "key_insight": "Mixtral 8x7B achieves strong performance with 6x faster inference using sparse mixture of experts.",
     "why_relevant": "Directly addresses inference optimization - core team interest."},
    
    {"paper_id": "2403.03507v2", "relevance_score": 8,
     "key_insight": "Claude 3 family shows strong multimodal performance with reduced incorrect refusals.",
     "why_relevant": "Important benchmark for frontier model capabilities."},
    
    {"paper_id": "2305.10601v2", "relevance_score": 9,
     "key_insight": "Tree of Thoughts enables deliberate problem-solving through exploration and backtracking.",
     "why_relevant": "Key prompting technique for complex reasoning tasks."},
    
    {"paper_id": "2210.03629v3", "relevance_score": 10,
     "key_insight": "ReAct synergizes reasoning and acting by interleaving thought traces with actions.",
     "why_relevant": "Foundational paper for AI agents - essential reading."},
    
    {"paper_id": "2305.14314v2", "relevance_score": 9,
     "key_insight": "QLoRA enables 65B model fine-tuning on single GPU with 4-bit quantization and LoRA.",
     "why_relevant": "Critical for efficient fine-tuning and deployment."},
    
    {"paper_id": "2307.09288v2", "relevance_score": 8,
     "key_insight": "Llama 2 provides strong open-source foundation models up to 70B parameters.",
     "why_relevant": "Important baseline model for the ecosystem."},
    
    {"paper_id": "2309.16609v2", "relevance_score": 9,
     "key_insight": "Mistral 7B outperforms larger models with efficient architecture design.",
     "why_relevant": "Demonstrates inference efficiency gains - core interest."},
    
    {"paper_id": "2305.18290v2", "relevance_score": 8,
     "key_insight": "DPO simplifies preference learning without explicit reward modeling or RL.",
     "why_relevant": "Practical fine-tuning technique, though more training-focused."},
    
    {"paper_id": "2402.03300v2", "relevance_score": 9,
     "key_insight": "Self-Discover enables LLMs to compose their own reasoning structures for complex tasks.",
     "why_relevant": "Advanced prompting technique for reasoning improvements."},
    
    {"paper_id": "2310.06825v2", "relevance_score": 9,
     "key_insight": "Self-RAG trains models to adaptively retrieve and self-critique for better factuality.",
     "why_relevant": "Practical approach to reducing hallucinations in production."},
    
    {"paper_id": "2401.02954v2", "relevance_score": 7,
     "key_insight": "DeepSeek LLM provides transparent open-source models up to 67B parameters.",
     "why_relevant": "Good open-source option but less novel techniques."},
    
    {"paper_id": "2308.12950v3", "relevance_score": 9,
     "key_insight": "Code Llama provides state-of-the-art open code generation with 100K context.",
     "why_relevant": "Directly applicable to AI-assisted coding workflows."},
    
    {"paper_id": "2312.11805v3", "relevance_score": 8,
     "key_insight": "Gemini demonstrates strong multimodal capabilities across text, image, audio, video.",
     "why_relevant": "Important frontier model benchmark."},
    
    {"paper_id": "2310.12931v2", "relevance_score": 7,
     "key_insight": "Llemma excels at mathematical reasoning and formal theorem proving.",
     "why_relevant": "Specialized model, less general applicability."},
    
    {"paper_id": "2309.05463v2", "relevance_score": 8,
     "key_insight": "phi-1.5 shows data quality matters more than quantity for small models.",
     "why_relevant": "Relevant insight for efficient model training."},
    
    {"paper_id": "2401.10020v1", "relevance_score": 10,
     "key_insight": "Comprehensive survey of resource-efficient techniques including compression and optimization.",
     "why_relevant": "Essential reading for inference optimization goals."},
    
    {"paper_id": "2403.08295v2", "relevance_score": 8,
     "key_insight": "SIMA demonstrates generalist agents that follow instructions across game environments.",
     "why_relevant": "Interesting for agent generalization, though game-focused."}
]

# Use mock results if we didn't run the API
if not RUN_BATCH_API or not API_KEY:
    evaluations = MOCK_EVALUATIONS
    print(f"✓ Using {len(evaluations)} mock evaluations")

### Parse the Results

If you ran the real API, we need to parse the JSON responses.
The mock results are already in the right format.

In [7]:
def parse_json_response(content: str) -> dict | None:
    """
    Extract JSON from the model's response.
    Handles cases where the model includes extra text.
    """
    content = content.strip()
    
    # Try direct parsing first
    if content.startswith('{'):
        try:
            return json.loads(content)
        except json.JSONDecodeError:
            pass
    
    # Try to find JSON anywhere in the response
    json_match = re.search(r'\{[\s\S]*\}', content)
    if json_match:
        try:
            return json.loads(json_match.group())
        except json.JSONDecodeError:
            pass
    
    return None


def parse_batch_results(raw_results: list[dict]) -> list[dict]:
    """
    Parse raw API results into evaluation dictionaries.
    """
    evaluations = []
    
    for result in raw_results:
        paper_id = result['custom_id']
        
        # Check for errors
        if result.get('error'):
            print(f"  ⚠️ Error for {paper_id}: {result['error']}")
            continue
        
        # Extract content from OpenAI response format
        response_body = result.get('response', {}).get('body', {})
        choices = response_body.get('choices', [])
        
        if not choices:
            continue
        
        content = choices[0].get('message', {}).get('content', '')
        evaluation = parse_json_response(content)
        
        if evaluation:
            evaluation['paper_id'] = paper_id
            evaluations.append(evaluation)
    
    return evaluations

# Parse real results if we have them
if RUN_BATCH_API and API_KEY:
    evaluations = parse_batch_results(raw_results)
    print(f"✓ Parsed {len(evaluations)} evaluations")

### Merge Results with Paper Data

Now let's combine the evaluations with the original paper data
so we have everything in one place.

In [16]:
def merge_results_with_papers(papers: list[dict], evaluations: list[dict]) -> list[dict]:
    """
    Combine paper data with evaluation results.
    Returns merged list sorted by relevance score (highest first).
    """
    
    # Create lookup for evaluations by paper_id
    eval_lookup = {e['paper_id']: e for e in evaluations}
    
    merged = []
    for paper in papers:
        paper_id = paper['id']
        if paper_id in eval_lookup:
            # Combine paper data with evaluation
            combined = {**paper, **eval_lookup[paper_id]}
            combined.pop('paper_id', None)  # Remove redundant field
            merged.append(combined)
    
    # Sort by relevance score (highest first)
    merged.sort(key=lambda x: x.get('relevance_score', 0), reverse=True)
    
    return merged

# Merge everything together
results = merge_results_with_papers(papers, MOCK_EVALUATIONS)
print(f"✓ Merged {len(results)} papers with evaluations")

✓ Merged 20 papers with evaluations


### Display Top 10 Papers

Finally! Let's see which papers the AI thinks our team should read.

In [17]:
# Display the top 10 most relevant papers
TOP_N = 10
MIN_SCORE = 7  # Only show papers with score >= 7

print("="*80)
print(f"TOP {TOP_N} PAPERS FOR YOUR TEAM (score >= {MIN_SCORE})")
print("="*80)

shown = 0
for paper in results:
    score = paper.get('relevance_score', 0)
    
    if score < MIN_SCORE:
        continue
    
    shown += 1
    if shown > TOP_N:
        break
    
    # Score indicator
    if score >= 9:
        indicator = "🔥"
    elif score >= 7:
        indicator = "⭐"
    else:
        indicator = "📄"
    
    print(f"\n{indicator} [{score}/10] {paper['title'][:65]}{'...' if len(paper['title']) > 65 else ''}")
    print(f"   💡 {paper.get('key_insight', 'N/A')}")
    print(f"   📎 {paper['url']}")

print(f"\n" + "="*80)
print(f"Showing {shown} papers with relevance score >= {MIN_SCORE}")

TOP 10 PAPERS FOR YOUR TEAM (score >= 7)

🔥 [10/10] ReAct Agents: Combining Reasoning and Acting in Language Models
   💡 Improved framework for AI agents that interleave reasoning with actions for better decision-making.
   📎 https://arxiv.org/abs/2401.00002

🔥 [10/10] Tool Use in Large Language Models: A Comprehensive Survey
   💡 Comprehensive survey of tool-augmented LLMs covering selection, invocation, and error recovery.
   📎 https://arxiv.org/abs/2401.00005

🔥 [10/10] Deployment Patterns for LLM Applications in Production
   💡 Catalog of deployment patterns for LLM applications covering batching, caching, and monitoring.
   📎 https://arxiv.org/abs/2401.00019

🔥 [9/10] Efficient Inference for Large Language Models: A Survey
   💡 Comprehensive survey of inference optimization techniques including quantization, pruning, and speculative decoding.
   📎 https://arxiv.org/abs/2401.00001

🔥 [9/10] Prompt Engineering Best Practices for Production Systems
   💡 Systematic study of prompt eng

---

## Step 3: Understanding the Results

Let's analyze what we got and discuss the approach.

### The JSON Structure

Each evaluated paper now contains both the original arXiv data
and the AI-generated evaluation.

In [18]:
# Look at the complete structure of a top paper
print("📊 Complete structure of a top-rated paper:")
print("="*80)

top_paper = results[0]
print(json.dumps({
    'id': top_paper['id'],
    'title': top_paper['title'][:50] + '...',
    'authors': top_paper['authors'][:2],
    'abstract': top_paper['abstract'][:100] + '...',
    'published': top_paper['published'],
    'url': top_paper['url'],
    'relevance_score': top_paper['relevance_score'],
    'key_insight': top_paper['key_insight'],
    'why_relevant': top_paper['why_relevant']
}, indent=2))

📊 Complete structure of a top-rated paper:
{
  "id": "2401.00002v1",
  "title": "ReAct Agents: Combining Reasoning and Acting in La...",
  "authors": [
    "David Park",
    "Emily Zhang"
  ],
  "abstract": "We present an improved framework for building AI agents that interleave reasoning traces with action...",
  "published": "2024-01-14T10:30:00Z",
  "url": "https://arxiv.org/abs/2401.00002",
  "relevance_score": 10,
  "key_insight": "Improved framework for AI agents that interleave reasoning with actions for better decision-making.",
  "why_relevant": "Highly relevant - AI agents and tool use is a primary team interest."
}


### Score Distribution

Let's see how the scores are distributed across all papers.

In [19]:
# Count papers in each score range
score_ranges = {
    'High (9-10)': 0,
    'Good (7-8)': 0,
    'Medium (4-6)': 0,
    'Low (0-3)': 0
}

for paper in results:
    score = paper.get('relevance_score', 0)
    if score >= 9:
        score_ranges['High (9-10)'] += 1
    elif score >= 7:
        score_ranges['Good (7-8)'] += 1
    elif score >= 4:
        score_ranges['Medium (4-6)'] += 1
    else:
        score_ranges['Low (0-3)'] += 1

print("📈 Score Distribution:")
print("="*40)
for range_name, count in score_ranges.items():
    bar = "█" * count
    print(f"{range_name:15} | {bar} ({count})")

# Average score
avg_score = sum(p.get('relevance_score', 0) for p in results) / len(results)
print(f"\nAverage score: {avg_score:.1f}")

📈 Score Distribution:
High (9-10)     | █████████ (9)
Good (7-8)      | ███████ (7)
Medium (4-6)    | ████ (4)
Low (0-3)       |  (0)

Average score: 7.8


### The Score Distribution Problem

Notice the average score is **7.8** - and most papers cluster between 7-9. This is a common problem with LLM evaluation!

The model is being "polite" and hedging - everything seems "pretty good." With such low variance, it's hard to prioritize which papers to actually read.

**We need external signals to differentiate!**

---

## Part 2: Adding External Signals (Citations)

The LLM gives us *relevance* to our team, but it's not the only signal. **Citation counts** tell us how the broader research community values each paper.

**Important for recent papers:** Since we're looking at papers from the last 7 days, citation counts will be LOW (typically 0-50). Even 10+ citations in a week indicates a paper is getting attention!

| LLM Score | Citations | Interpretation |
|-----------|-----------|----------------|
| High | High (for age) | **Must read** - relevant AND gaining traction |
| High | Low | New/niche - relevant but hasn't spread yet |
| Low | High | Important to others, but not for us |
| Low | Low | Skip |

Let's add citation data using Semantic Scholar's free API.

In [24]:
# Import our tools for citation lookup
from tools import check_citation_impact, get_citations_batch, create_composite_score

# For workshop demo, we'll use pre-computed citation data
# (Real API calls take ~10 seconds for 20 papers due to rate limiting)

USE_LIVE_CITATIONS = True  # Set True to fetch real citation data

if USE_LIVE_CITATIONS:
    print("📚 Fetching citation data from Semantic Scholar...")
    results_with_citations = get_citations_batch(results)
else:
    # Mock citation data - REALISTIC for recent papers (last 7 days)
    # New papers typically have 0-50 citations, not hundreds!
    MOCK_CITATIONS = {
        "2401.00001v1": 28,    # Survey - getting attention
        "2401.00002v1": 45,    # ReAct follow-up - popular topic
        "2401.00003v1": 12,    # Practical guide
        "2401.00004v1": 8,     # Scaling laws
        "2401.00005v1": 33,    # Tool use - hot topic
        "2401.00006v1": 19,    # RAG improvements
        "2401.00007v1": 15,    # Constitutional AI
        "2401.00008v1": 22,    # Multi-agent
        "2401.00009v1": 31,    # Quantization - practical
        "2401.00010v1": 9,     # Evaluation
        "2401.00011v1": 6,     # Long context
        "2401.00012v1": 14,    # Code gen guide
        "2401.00013v1": 38,    # Speculative decoding - efficiency!
        "2401.00014v1": 7,     # Instruction following
        "2401.00015v1": 11,    # MoE
        "2401.00016v1": 3,     # Theory of mind - niche
        "2401.00017v1": 5,     # Safety alignment
        "2401.00018v1": 16,    # In-context learning
        "2401.00019v1": 21,    # Deployment patterns
        "2401.00020v1": 4,     # Architecture ablation
    }
    
    results_with_citations = []
    for paper in results:
        paper_with_cit = {**paper}
        paper_with_cit['citations'] = MOCK_CITATIONS.get(paper['id'], 0)
        paper_with_cit['found'] = True
        results_with_citations.append(paper_with_cit)
    
    print("✓ Using mock citation data (realistic for recent papers: 0-50 range)")

📚 Fetching citation data from Semantic Scholar...
  [1/20] Checking: ReAct Agents: Combining Reasoning and Acting in La...
  [2/20] Checking: Tool Use in Large Language Models: A Comprehensive...
  [3/20] Checking: Deployment Patterns for LLM Applications in Produc...
  [4/20] Checking: Efficient Inference for Large Language Models: A S...
  [5/20] Checking: Prompt Engineering Best Practices for Production S...
  [6/20] Checking: Multi-Agent Collaboration with Large Language Mode...
  [7/20] Checking: Quantization Techniques for Efficient LLM Deployme...
  [8/20] Checking: Code Generation with Large Language Models: A Prac...
  [9/20] Checking: Speculative Decoding: Accelerating LLM Inference W...
  [10/20] Checking: Retrieval-Augmented Generation: Reducing Hallucina...
  [11/20] Checking: Evaluating Large Language Models: Challenges and B...
  [12/20] Checking: Instruction Following in Large Language Models...
  [13/20] Checking: In-Context Learning: Understanding How LLMs Learn ...
 

In [25]:
# Compare LLM scores vs Citation counts
print("📊 LLM SCORES vs CITATION COUNTS")
print("=" * 80)
print(f"{'Paper':<45} | {'LLM':>5} | {'Citations':>10}")
print("-" * 80)

# Sort by citations to see the difference
sorted_by_citations = sorted(results_with_citations, key=lambda x: x['citations'], reverse=True)

for paper in sorted_by_citations[:10]:
    title = paper['title'][:42] + "..." if len(paper['title']) > 42 else paper['title']
    print(f"{title:<45} | {paper['relevance_score']:>5} | {paper['citations']:>10,}")

print()
print("💡 KEY INSIGHT: Citation counts have MUCH higher variance!")
print("   Range: {} to {:,} (vs LLM scores: {} to {})".format(
    min(p['citations'] for p in results_with_citations),
    max(p['citations'] for p in results_with_citations),
    min(p['relevance_score'] for p in results_with_citations),
    max(p['relevance_score'] for p in results_with_citations)
))

📊 LLM SCORES vs CITATION COUNTS
Paper                                         |   LLM |  Citations
--------------------------------------------------------------------------------
Prompt Engineering Best Practices for Prod... |     9 |         12
Mixture of Experts for Efficient Large Lan... |     7 |          9
Code Generation with Large Language Models... |     9 |          6
Speculative Decoding: Accelerating LLM Inf... |     9 |          3
In-Context Learning: Understanding How LLM... |     8 |          3
ReAct Agents: Combining Reasoning and Acti... |    10 |          2
Tool Use in Large Language Models: A Compr... |    10 |          2
Efficient Inference for Large Language Mod... |     9 |          2
Multi-Agent Collaboration with Large Langu... |     9 |          2
Long Context Understanding in Transformer ... |     7 |          2

💡 KEY INSIGHT: Citation counts have MUCH higher variance!
   Range: 0 to 12 (vs LLM scores: 4 to 10)


---

## Part 3: Intelligent Re-ranking with Composite Scores

Now let's combine both signals:
- **LLM Score** (70%) - Is this relevant to OUR team?
- **Citations** (30%) - Does the research community value this?

This gives us a composite score that balances **relevance** with **impact**.

In [23]:
from tools import add_composite_scores, compare_rankings

# Add composite scores (70% LLM, 30% citations)
results_with_composite = add_composite_scores(
    results_with_citations,
    llm_weight=0.7,
    citation_weight=0.3
)

print("📊 PAPERS WITH COMPOSITE SCORES")
print("=" * 90)
print(f"{'Paper':<40} | {'LLM':>5} | {'Citations':>10} | {'Composite':>10}")
print("-" * 90)

for paper in results_with_composite[:10]:
    title = paper['title'][:37] + "..." if len(paper['title']) > 37 else paper['title']
    print(f"{title:<40} | {paper['relevance_score']:>5} | {paper['citations']:>10,} | {paper['composite_score']:>10.2f}")

   (Auto-detected max_citations=20 based on data)
📊 PAPERS WITH COMPOSITE SCORES
Paper                                    |   LLM |  Citations |  Composite
------------------------------------------------------------------------------------------
Prompt Engineering Best Practices for... |     9 |         12 |       8.83
Deployment Patterns for LLM Applicati... |    10 |          2 |       8.08
Quantization Techniques for Efficient... |     9 |          3 |       7.67
Efficient Inference for Large Languag... |     9 |          2 |       7.38
Multi-Agent Collaboration with Large ... |     9 |          2 |       7.38
ReAct Agents: Combining Reasoning and... |    10 |          0 |       7.00
Tool Use in Large Language Models: A ... |    10 |          0 |       7.00
Code Generation with Large Language M... |     9 |          0 |       6.30
Speculative Decoding: Accelerating LL... |     9 |          0 |       6.30
Retrieval-Augmented Generation: Reduc... |     8 |          0 |       5.60


In [ ]:
# Compare the three ranking strategies side-by-side
rankings = compare_rankings(results_with_composite, top_n=10)

print("\n🔄 RANKING COMPARISON: How do different strategies rank papers?")
print("=" * 105)
print(f"{'#':<3} | {'LLM-Only Ranking':<32} | {'Citation-Only Ranking':<32} | {'Composite Ranking':<32}")
print("-" * 105)

for i in range(5):
    llm = rankings['llm_ranking'][i]
    cit = rankings['citation_ranking'][i]
    comp = rankings['composite_ranking'][i]
    
    llm_str = f"{llm['title'][:22]}... (LLM:{llm['relevance_score']})"
    cit_str = f"{cit['title'][:22]}... ({cit['citations']:,})"
    comp_str = f"{comp['title'][:22]}... ({comp['composite_score']:.1f})"
    
    print(f"{i+1:<3} | {llm_str:<32} | {cit_str:<32} | {comp_str:<32}")

print("-" * 105)
print()
print("💡 Notice how the COMPOSITE ranking surfaces different papers than either alone!")
print("   It balances 'relevant to us' with 'valued by the community'.")

---

## Part 4: Intelligent Tool Use

Now we can make **intelligent decisions** about when to use expensive tools.

Instead of downloading PDFs for all papers (slow, wasteful), the agent reasons:
> "This paper is BOTH highly relevant AND highly valued by the community.
> It's worth the cost of downloading and analyzing the full PDF."

This is **agentic behavior** - using judgment to decide when to take action.

In [ ]:
from tools import fetch_paper_pdf

# Define thresholds for "high value" papers
# For RECENT papers (last 7 days), citations will be low (0-50 typically)
# Adjust thresholds based on your data!
COMPOSITE_THRESHOLD = 8.0   # Top ~20% of papers
CITATION_THRESHOLD = 5      # Even 5 citations in a week is notable for new papers

print("🤖 AGENT REASONING: Which papers deserve deep investigation?")
print("=" * 70)
print(f"Criteria: composite_score >= {COMPOSITE_THRESHOLD} AND citations >= {CITATION_THRESHOLD}")
print("(Thresholds adjusted for RECENT papers - new papers have few citations)")
print()

high_value_papers = []

for paper in results_with_composite:
    composite = paper['composite_score']
    citations = paper['citations']
    llm_score = paper['relevance_score']
    
    if composite >= COMPOSITE_THRESHOLD and citations >= CITATION_THRESHOLD:
        high_value_papers.append(paper)
        
        print(f"🔥 HIGH VALUE: {paper['title'][:55]}...")
        print(f"   └─ Composite: {composite:.1f} | LLM: {llm_score} | Citations: {citations:,}")
        print(f"   └─ Agent reasoning: 'Highly relevant AND gaining traction quickly'")
        print()

if not high_value_papers:
    print("No papers met both thresholds. Try lowering COMPOSITE_THRESHOLD or CITATION_THRESHOLD.")
else:
    print(f"Found {len(high_value_papers)} papers worth deep investigation.")

In [ ]:
# Optionally download PDFs for high-value papers
DOWNLOAD_PDFS = False  # Set True to actually download

if DOWNLOAD_PDFS and high_value_papers:
    print("📥 Downloading PDFs for high-value papers...")
    for paper in high_value_papers[:3]:  # Limit to top 3
        print(f"📄 Downloading: {paper['title'][:50]}...")
        pdf_path = fetch_paper_pdf(paper['id'], output_dir="./papers")
        if pdf_path:
            print(f"   ✓ Saved to: {pdf_path}")
        print()
else:
    print("📋 Papers ready for download (set DOWNLOAD_PDFS = True to fetch):")
    for i, paper in enumerate(high_value_papers[:3], 1):
        print(f"   {i}. {paper['title'][:60]}...")
        print(f"      URL: https://arxiv.org/pdf/{paper['id'].replace('v1', '')}.pdf")

---

## Summary: From Batch Processing to Intelligent Agents

We've evolved our system through several stages:

| Stage | Approach | Limitation |
|-------|----------|------------|
| **Part 1** | LLM batch evaluation | Scores cluster around 7-8, hard to prioritize |
| **Part 2** | Add citation signals | Now have two independent metrics |
| **Part 3** | Composite scoring | Combine relevance + impact for better ranking |
| **Part 4** | Tool orchestration | Agent decides WHEN to use expensive tools |

### Key Takeaways

1. **LLMs are "polite"** - They tend to give middle-ground scores. External signals add variance.

2. **Multiple signals > single signal** - Combining LLM judgment with community metrics gives better rankings.

3. **Agents make decisions** - Instead of running every tool on every input, agents reason about when tools are worth using.

### What's Next?

In the next part, we'll give the LLM the ability to:
- Choose which tools to call autonomously
- Decide what to do based on results  
- Iterate until the task is complete
- Handle errors and recover

### Papers That Didn't Make the Cut

Let's also look at low-scoring papers to understand the AI's reasoning.

In [29]:
# Show papers with low scores
print("📉 Low-Scoring Papers (why weren't they relevant?):")
print("="*80)

low_scoring = [p for p in results if p.get('relevance_score', 0) < 7]

for paper in low_scoring[:5]:
    score = paper.get('relevance_score', 0)
    print(f"\n[{score}/10] {paper['title'][:60]}...")
    print(f"   Why: {paper.get('why_relevant', 'N/A')}")

📉 Low-Scoring Papers (why weren't they relevant?):

[6/10] Safety Alignment in Large Language Models: Methods and Chall...
   Why: Background knowledge but not directly applicable to current work.

[5/10] Scaling Laws for Fine-tuning Language Models...
   Why: Useful but focused on training dynamics rather than inference optimization.

[5/10] Advanced Topics in Transformer Architecture Design...
   Why: More focused on architecture design than practical application.

[4/10] Theory of Mind in Large Language Models: An Empirical Invest...
   Why: Interesting but more theoretical than practical for the team.


### Discussion: What Worked / What Didn't

**What worked well:**
- ✅ Papers about practical topics (agents, inference, deployment) scored high
- ✅ The AI correctly identified papers matching our "interests" list
- ✅ Papers flagged in "avoid" (pure theory, training-focused) scored lower

**Limitations of this approach:**
- ❌ **No iteration** - we can't ask follow-up questions
- ❌ **No context** - each paper is evaluated independently
- ❌ **No actions** - can't download PDFs, read full text, etc.
- ❌ **Fixed criteria** - can't adjust mid-evaluation

**This is why we need agents!**

In Part 2, we'll give the AI tools to:
- Search for related papers
- Download and read full PDFs
- Summarize findings
- Decide what to do next

---

## Exercises

Try these modifications to better understand the system:

### Exercise 1: Change the Team Profile

Modify `TEAM_PROFILE` above to match a different team's interests.
For example, a team focused on healthcare AI or computer vision.
Then re-run the evaluation to see how scores change.

In [ ]:
# Example: Healthcare AI team profile
HEALTHCARE_TEAM_PROFILE = {
    "focus": """
        The team builds AI systems for healthcare applications including
        medical imaging analysis, clinical decision support, and patient
        communication tools.
    """,
    
    "interests": [
        "Medical image analysis and diagnosis",
        "Clinical NLP and medical text understanding",
        "Privacy-preserving machine learning",
        "Explainable AI for healthcare",
        "Patient-facing AI applications",
    ],
    
    "avoid": [
        "General-purpose language models without medical focus",
        "Papers without clinical validation",
        "Purely theoretical work",
    ]
}

# Try using this profile instead!
# Uncomment the line below and re-run the batch evaluation
# TEAM_PROFILE = HEALTHCARE_TEAM_PROFILE

### Exercise 2: Adjust the Scoring Criteria

Modify `create_evaluation_prompt()` to use different scoring criteria.
For example, add factors like "novelty" or "implementation difficulty".

### Exercise 3: Fetch Live Papers

Set `USE_LIVE_DATA = True` and `RUN_BATCH_API = True` to:
1. Fetch real papers from arXiv
2. Evaluate them with the actual Batch API

Compare the results with the sample data!

---

## Summary

In Part 1, we built a **baseline** paper evaluation system:

1. **Fetch** - Retrieved 20 papers from arXiv
2. **Batch** - Created JSONL file with evaluation requests
3. **Submit** - Sent to the Batch API
4. **Parse** - Extracted structured JSON from responses
5. **Rank** - Sorted papers by relevance score

**Key insight:** This is a fixed pipeline. The AI evaluates papers but can't:
- Ask clarifying questions
- Decide to search for more papers
- Read full paper text
- Take any actions beyond scoring

**Next up in Part 2:** We'll add tools and turn this into a true AI agent!

In [ ]:
# Clean up the batch file
import os
if os.path.exists('batch_input.jsonl'):
    os.remove('batch_input.jsonl')
    print("✓ Cleaned up batch_input.jsonl")